<a href="https://colab.research.google.com/github/gerbeldo/2022-dma-final/blob/main/labo2_petfinder_final_%5Bbeldorati-bondarchuk-brankevich-varise%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Petfinder

**Grupo: Germán Beldorati Stark, Maria Bondarchuk, Paula Brankevich, Florencia Varise**

El objetivo del trabajo es la creación de un modelo de clasificación para predecir la velocidad de adopción de mascotas usando la base de datos *Petfinder*. En este *script*, preprocesamos los datos, definimos un modelo, optimizamos algunos de sus hiperparámetros, lo entrenamos y generamos predicciones sobre el *dataset* de *test*.

Importamos las librerías necesarias.

In [2]:
%%capture
!pip install optuna
import optuna
import optuna.integration.lightgbm as lgb
import numpy as np
import pandas as pd
import pprint
from sklearn.model_selection import KFold, train_test_split

Cargamos datos de training y testing, alojados en github por comodidad.

In [3]:
data = pd.read_csv("https://github.com/gerbeldo/2022-dma-final/blob/main/train.csv.zip?raw=true", compression = "zip")
test = pd.read_csv("https://github.com/gerbeldo/2022-dma-final/blob/main/test.csv.zip?raw=true", compression = "zip")

Definimos una función que lleva a cabo todos los pasos de limpieza de datos según lo estudiado en el [script de EDA](https://github.com/gerbeldo/2022-dma-final/blob/main/0_EDA.ipynb) (alojado en github), la cual resumimos a continuación:

En primer lugar, una corrección de los tipos de datos, luego la eliminación de variables con alta proporción de *missing values*, la creación de una nueva variable ("has_name") para sacarnos de encima la alta cardinalidad de los nombres, y por último la eliminación de las variables ID y *strings*.

In [4]:
def preproceso(data):
    """elimina variables innecesarias, genera nuevas, corrige tipos de datos"""

    # correccion de tipo
    data = data.astype({'PhotoAmt' : int})

    # eliminamos variables con alta proporcion de NAs
    data = data.drop(["Breed2", "Color3"], axis = 1)
    
    # nueva variable
    data['Name'] = data['Name'].fillna('Unnamed')
    data['Has_name'] = 1
    data.loc[data['Name'] == 'Unnamed', 'Has_name'] = 0
    data = data.drop("Name", axis = 1)

    # eliminar variables string
    data = data.drop(["Description", "PetID", "RescuerID"], axis = 1)

    return data

Preprocesamos tanto el dataset de *training* como el de *testing*. Además, transformamos el dataset de *train* al formato necesario para LGMB.

In [5]:
data_pre = preproceso(data)
test_pre = preproceso(test)

X = data_pre.drop("AdoptionSpeed", axis = 1)
y = data_pre["AdoptionSpeed"]

dtrain = lgb.Dataset(X, label=y)

Definimos variables categóricas, para especificárselas a lightGBM.

In [6]:
cat_vars = ['Type',
            'Breed1',
            'Gender',
            'Color1',
            'Color2',
            'MaturitySize',
            'FurLength',
            'Vaccinated',
            'Dewormed',
            'Sterilized',
            'Health',
            'State',
            'Has_name']

Creamos los *folds* para la optimización (10-fold CV).

In [8]:
folds = KFold(n_splits=10, shuffle=True, random_state=42)

Para la optimización de hiperparámetros utilizamos la libreria `optuna`. Es una libreria de optimización bayesiana, con la infraestructura necesaria para optimizar cualquier modelo, pero tiene la ventaja de que para algunos modelos muy utilizados (como LGBM) posee funciones de optimización específicas. Los parámetros que optimiza `LightGBMTunerCV` son: `feature_fraction`, `num_leaves`, `bagging_fraction`,  `bagging_freq` los factores de regularización (`lambda_l1` y `lambda_l2`) y `min_child_samples`.


In [9]:
# definimos algunos parametros fijos
params = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class" : 5,
    "verbosity": -1,
    "boosting_type": "gbdt",
    "seed": 42
    }

# creamos el estudio de optuna
study_tuner = optuna.create_study(direction="minimize")

# apagamos un poco los logs
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# definimos el tuner para LGBM
tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            categorical_feature = cat_vars,
                            study = study_tuner,
                            verbose_eval = False,                            
                            early_stopping_rounds = 50,
                            time_budget = 18000, # 5 horas
                            seed = 42,
                            folds = folds,
                            num_boost_round = 1000)

# corremos la optimización
tuner.run()

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
feature_fraction, val_score: 1.337426:  14%|#4        | 1/7 [00:12<01:17, 12.86s/it]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
feature_fraction, val_score: 1.337426:  29%|##8       | 2/7 [00:36<01:37, 19.49s/it]/usr/local/

Imprimimos el diccionario con los mejores parámetros encontrados, y los extraemos a una variable.

In [16]:
best_params = tuner.best_params
pprint.pprint(best_params)

{'bagging_fraction': 0.9004101632052598,
 'bagging_freq': 2,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.5,
 'feature_pre_filter': False,
 'lambda_l1': 0.024458537988108132,
 'lambda_l2': 8.54898728597986,
 'metric': 'multi_logloss',
 'min_child_samples': 10,
 'num_class': 5,
 'num_leaves': 55,
 'objective': 'multiclass',
 'seed': 42,
 'verbosity': -1}


LightGBM requiere un set de validación para el entrenamiento, por lo que separamos 10% del dataset y entrenamos.

In [19]:
X_train, X_val, y_train, y_val = train_test_split(data_pre.drop("AdoptionSpeed", axis = 1),
                                                  data_pre["AdoptionSpeed"],
                                                  test_size = 0.10,
                                                  random_state = 42)

dtrain = lgb.Dataset(X_train, label = y_train)
dval = lgb.Dataset(X_val, label = y_val, reference = dtrain)

# utilizamos los mejores parámetros
lgbfit = lgb.train(best_params,
                   dtrain,
                   valid_sets = [dtrain, dval],
                   valid_names = ["training", "validation"],
                   categorical_feature=cat_vars,
                   verbose_eval=10,                   
                   num_boost_round=1000)

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's multi_logloss: 1.35719	validation's multi_logloss: 1.39553
[20]	training's multi_logloss: 1.29619	validation's multi_logloss: 1.36325
[30]	training's multi_logloss: 1.2542	validation's multi_logloss: 1.34667
[40]	training's multi_logloss: 1.21933	validation's multi_logloss: 1.33721
[50]	training's multi_logloss: 1.19082	validation's multi_logloss: 1.33175
[60]	training's multi_logloss: 1.16475	validation's multi_logloss: 1.32838
[70]	training's multi_logloss: 1.14183	validation's multi_logloss: 1.32687
[80]	training's multi_logloss: 1.12061	validation's multi_logloss: 1.32696
[90]	training's multi_logloss: 1.10088	validation's multi_logloss: 1.3278
[100]	training's multi_logloss: 1.0827	validation's multi_logloss: 1.32862
[110]	training's multi_logloss: 1.06505	validation's multi_logloss: 1.32902
[120]	training's multi_logloss: 1.04827	validation's multi_logloss: 1.33019
[130]	training's multi_logloss: 1.03257	validation's multi_logloss: 1.33172
[140]	training's multi_l

feature_fraction, val_score: 1.498663:  14%|#4        | 1/7 [00:29<02:56, 29.45s/it]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's multi_logloss: 1.3664	validation's multi_logloss: 1.40274
[20]	training's multi_logloss: 1.30657	validation's multi_logloss: 1.36889
[30]	training's multi_logloss: 1.26689	validation's multi_logloss: 1.35335
[40]	training's multi_logloss: 1.23229	validation's multi_logloss: 1.34227
[50]	training's multi_logloss: 1.20468	validation's multi_logloss: 1.33595
[60]	training's multi_logloss: 1.18007	validation's multi_logloss: 1.33257
[70]	training's multi_logloss: 1.15839	validation's multi_logloss: 1.33094
[80]	training's multi_logloss: 1.13808	validation's multi_logloss: 1.33172
[90]	training's multi_logloss: 1.11958	validation's multi_logloss: 1.33199
[100]	training's multi_logloss: 1.10185	validation's multi_logloss: 1.33261
[110]	training's multi_logloss: 1.08513	validation's multi_logloss: 1.33276
[120]	training's multi_logloss: 1.06898	validation's multi_logloss: 1.33489
[130]	training's multi_logloss: 1.05423	validation's multi_logloss: 1.33647
[140]	training's multi

feature_fraction, val_score: 1.493265:  29%|##8       | 2/7 [00:57<02:24, 28.85s/it]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


...

min_data_in_leaf, val_score: 1.337978:  60%|######    | 3/5 [00:29<00:19,  9.93s/it]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's multi_logloss: 1.42201	validation's multi_logloss: 1.43172
[20]	training's multi_logloss: 1.39837	validation's multi_logloss: 1.41018
[30]	training's multi_logloss: 1.38448	validation's multi_logloss: 1.39728
[40]	training's multi_logloss: 1.36994	validation's multi_logloss: 1.38526
[50]	training's multi_logloss: 1.35986	validation's multi_logloss: 1.37768
[60]	training's multi_logloss: 1.35152	validation's multi_logloss: 1.37069
[70]	training's multi_logloss: 1.34543	validation's multi_logloss: 1.36569
[80]	training's multi_logloss: 1.34051	validation's multi_logloss: 1.3616
[90]	training's multi_logloss: 1.33625	validation's multi_logloss: 1.35851
[100]	training's multi_logloss: 1.33232	validation's multi_logloss: 1.35629
[110]	training's multi_logloss: 1.32904	validation's multi_logloss: 1.35414
[120]	training's multi_logloss: 1.32622	validation's multi_logloss: 1.35176
[130]	training's multi_logloss: 1.32361	validation's multi_logloss: 1.3497
[140]	training's multi_

min_data_in_leaf, val_score: 1.337978:  80%|########  | 4/5 [00:39<00:09,  9.93s/it]/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Breed1', 'Color1', 'Color2', 'Dewormed', 'FurLength', 'Gender', 'Has_name', 'Health', 'MaturitySize', 'State', 'Sterilized', 'Type', 'Vaccinated']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's multi_logloss: 1.42201	validation's multi_logloss: 1.43172
[20]	training's multi_logloss: 1.39837	validation's multi_logloss: 1.41018
[30]	training's multi_logloss: 1.38448	validation's multi_logloss: 1.39728
[40]	training's multi_logloss: 1.36994	validation's multi_logloss: 1.38526
[50]	training's multi_logloss: 1.35986	validation's multi_logloss: 1.37768
[60]	training's multi_logloss: 1.35152	validation's multi_logloss: 1.37069
[70]	training's multi_logloss: 1.34543	validation's multi_logloss: 1.36569
[80]	training's multi_logloss: 1.34051	validation's multi_logloss: 1.3616
[90]	training's multi_logloss: 1.33625	validation's multi_logloss: 1.35851
[100]	training's multi_logloss: 1.33232	validation's multi_logloss: 1.35629
[110]	training's multi_logloss: 1.32904	validation's multi_logloss: 1.35414
[120]	training's multi_logloss: 1.32622	validation's multi_logloss: 1.35176
[130]	training's multi_logloss: 1.32361	validation's multi_logloss: 1.3497
[140]	training's multi_

min_data_in_leaf, val_score: 1.337978: 100%|##########| 5/5 [00:49<00:00,  9.95s/it]


Predecimos sobre el *dataset* de *test*, y colapsamos las predicciones  (tenemos 5 listas, con las probabilidades de cada una de las clases) en la predicción de clase, tomando como predicha la clase de máxima probabilidad por elemento.

In [20]:
preds = lgbfit.predict(test_pre)

class_predictions = []

for x in preds:
    class_predictions.append(np.argmax(x))

Guardamos las predicciones y los petIDs en un csv, listo para enviar a Kaggle.

In [21]:
submission = pd.DataFrame({"PetID" : test.PetID,
                           "AdoptionSpeed" : class_predictions})
submission.to_csv("./submission.csv", index = False)